# MUM 2023-24 Optymalizator

## Sieć neuronowa (NN)
<img style="float: right;" src="ml_figures/simple_nn.png" width=450>

* NN jest wielowymiarową funkcją (zwykle) nieliniową $F:X\longrightarrow Y$
  * dla funkcji zdefiniowana jest różniczkowalna funkcja kosztu $L:Y\times F(X)\longrightarrow \mathbb{R}$
  * uczenie odbywa się w pętli dla optymalizacji parametrów
  * problem MNIST (niespecjalnie dobre rozwiązanie!)
* w Pytorch sieć neuronowa jest zrealizowana jako [__graf obliczeniowy__](./12_Graf_obliczen.ipynb)
<img style="float: right;" src="ml_figures/comp_graph.png" width=400>

```python
class NeuralNetwork(nn.Module):    
    # (kod ze tutoriala `Pytorch`)
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
```
  * do tego metoda `forward` obliczająca wartość aktywacji
```python
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
```
  * `model = NeuralNetwork()` utworzy obiekt tej sieci
  * pętla ucząca
  ```python
  def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

  ```
  * i wykonanie
  ```python
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

  epochs = 10
  for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
  ```
  * funkcja kosztu `loss_fn` musi brać pod uwagę, jakie wartości zwracane są przez model
    * tutaj model zwraca `logits` z przedziału $[-\infty, +\infty]$, stąd `CrossEntropyLoss(x, y)`
    $$L(x, y) = -\sum_k\log\frac{\exp x_k}{\sum_i\exp x_i}y_k$$
  * predykcja modelu
    ```python
    logits = model(X)
    pred_proba = nn.Softmax(dim=1)(logits)
    y_pred = pred_proba.argmax(1)
    ```

# __Gradientowa__ i __iteracyjna__ minimalizacja $L$
* $w$ - wektor wszystkich parametrów modelu rozpoczynając od $w_0$
* $w_t$ - wartość parametrów po $t$ krokach (po czasie $t$)
* krok w czasie $t$ $\Delta _t := w_t-w_{t-1}$
* $L$ - _różniczkowalna_ funkcja kosztu
* $\gamma$ - __learning rate__
    * kontroluje szybkość uczenia
    * może zmieniać się w czasie $\gamma_t$
* Wartość $L$ zależy od:
  * danych treningowych i wektora $\theta$
  * architektury modelu
  * być może jeszcze innych stałych
* Optymalizujemy tylko $w$, będziemy pisać w skrócie $L(w_t)$.
* $L$ zwraca wartość: zakładamy, że umiemy policzyć gradient $\nabla L(w)$.

<!-- # Materiały dodatkowe -->

<!-- http://ruder.io/optimizing-gradient-descent/index.html -->
<!--  -->

## 😏Gradient Descent

### Update
<img style="float: right;" src="ml_figures/Optimizery_steepest_descent.png" width=400>

1. $w_{t+1}=\theta_{t} - \gamma\nabla L(w_{t})$
2. $-\nabla L(w_t)$ to kierunek najszybszego spadku $L$
  * jednak $-\nabla L(w_t)$ __nie wskazuje__ optymalnego kierunku $w$
3. wektor gradientu $\nabla L(w_t)$ jest prostopadły do hiperpłaszczyzny stycznej do powierzchni o równych wartościach funkcji kosztu (_isosurface_) w miejscu $w_t$
4. zmniejszenie $L$ odpowiada __co najmniej__ rozwartemu kątowi między $\nabla\theta^{(t)}$ a $\Delta\theta^{(t)}$
5. learning rate $\gamma$
    * mała wartość — spowalnia uczenie
    * duża wartość — powoduje __oscylacje__, problem ze zbieżnością
6. dlaczego nie normalizujemy $\nabla L(w_t)$?
    * duża wartość gradientu to __duża lokalna zmienność $L$__ - powinniśmy robić __mniejsze kroki__ (precyzyjniej)
    * mała wartość gradientu to __mała lokalna zmienność $L$__ - powinniśmy robić __większe kroki__ (aby szybciej opuścić __plateau__)
7. wariant GD ze zmiennym w czasie $\gamma$
  * malejący w stosunku odwrotnym do kroku uczenia $t$
    * warunki konieczne osiągnięcia optymalnego $\gamma$ (przy jakich założeniach? uwaga na __lokalne minima__)
        $$\begin{align}\sum_t\gamma_t^2\lt\infty\\\sum_t\gamma_t=\infty\end{align}$$
8. niech $\gamma_{opt}$ będzie _optymalna_
  * $\gamma<\gamma_{opt}$: uczenie będzie postępować zbyt małymi krokami
  * $\gamma=\gamma_{opt}$: uczenie powinno być jednokrokowe (funkcja $L$ wypukła)
  * $\gamma>\gamma_{opt}$: duże oscylacje uczenia jednak zbieżne do minimum
  * $\gamma\geq2\gamma_{opt}$: niebezpieczeństwo wyskoczenia i rozbieżności w uczeniu
9. zwykle jednak w praktyce najszybsza zbieżność jest osiągana przy $\gamma$ bliskim tej dla rozbieżności
  * należy używać jak największej $\gamma$ ale nie większej ;-)

__GD jest najgorszą możliwą metodą optymalizacji__ 😏 😒 😩 😒 😞

## Stochastic gradient descent SGD
$$w_{t+1}=w_t-\gamma_t\nabla{}L_i(w_t)$$
gdzie indeks $i$ wskazuje na wybrany losowo przykład (z rozkładu równomiernego)
* to __najlepsza metoda optymalizacji__ dlaczego?!?!
* można spojrzeć na SGD jako GD z szumem
  * przy uczeniu wielu przykładów jest duży stopień _redundancji_ przykładów, chociażby wiele tych samych cyfr w MNIST
  * w początkowych krokach szum jest niewielki w porównaniu do informacji w gradiencie
    * krok SGD jest równie dobry jak krok GD
  * szum może zabezpieczyć przed wpadnięciem do lokalnego (niepoprawnego) minimum
    * to przypomina _annealing_ - uczenie z _explicite_ dodanym stochastycznym szumem malejącym wraz z postępem uczenia
  * SGD jest drastycznie szybsze od GD i można wykonać tym samym kosztem tysiące kroków zamiast jednego kroku GD

### Hiperparametr stałej uczenia
* optymalna wartość stałej uczenia zależy od wielkości zbioru uczącego,
* a także od od wielkości batchu

[wizualizacja [deeplearning.ai]](https://www.deeplearning.ai/ai-notes/optimization/index.html)

## Mini batches
$$w_{t+1}=w_t-\gamma_t\frac{1}{|B_i|}\sum_{i\in{}B_i}\nabla{}L_i(w_t)$$
* istotny jest element $1/|B_i|$ 
* w oczywisty sposób można lepiej wykorzystać sprzęt, w szczególności GPU
* bardzo przydatne dla urównoleglenia uczenia
  

## Momentum
<img style="float: right;" src="ml_figures/momentum-nag.png" width=450>

* SGD z momentum to SGD z dodanym efektem ciężkiej kuli podążającej dotychczasowym kierunkiem
$$\begin{align*}
p_{t+1} &= \nabla L_i(w_t) + \beta p_t\\
w_{t+1}&=w_t - \gamma{}p_{t+1}=w_t - \gamma{}\nabla L_i(w_t) - \gamma\beta{}p_t\\
&\textrm{ponieważ}\\
p_t&=\nabla L_i(w_{t-1}) + \beta p_{t-1}\\
w_t&=w_{t-1} - \gamma{}p_t\\
-\gamma{}p_t&=w_t-w_{t-1}\\
&\textrm{stąd}\\
w_{t+1}&=w_t - \gamma_t\nabla L_i(w_t) + \beta(w_t-w_{t-1})
\end{align*}$$
gdzie ostatni składnik jest dodatkowy względem SGD
<img style="float: right;" src="ml_figures/opt2.gif" width=450>

0. Krok staje się kombinacją poprzedniego kierunku i negatywnego gradientu
1. Analogia do kulki toczącej się ze wzgórza, $\beta$ to tarcie lub opór powietrza
2. __Pamięć__
    * wzajemnie wzmacniają się kroki w __istotnym kierunku__ i kierunek nie jest zmieniany natychmiast
    * __oscylacje__ są tłumione i uśredniają się do małej wartości
    * mniejsze spowolnienie na __plateau__, jeśli "kulka" była rozpędzona
3. wartości hiperparametru $\beta=0.9$ lub $\beta=0.99$ sprawdzają się prawie zawsze
   * zwykle zwiększenie $\beta$ powinno skutkować _zmniejszeniem_ $\gamma$ dla utrzymania zbieżności

To jest ten _free lunch_, którego podobno niema 😏

## Nesterov accelerated gradient NAG

  * NAG jest wyjściem o jeden krok do przodu i wzięciem wartości w miejscu, gdzie optymalizator dopiero się znajdzie
$$\begin{align}
p_{t+1} &= \nabla L_i(w_t) +  \beta{}p_t\\
w_{t+1}&=w_t - \gamma_t(\nabla L_i(w_t)+\beta{}p_{t+1})\\
       &=w_t - \gamma_t(1+\beta)\nabla L_i(w_t) - \gamma_t\beta^2p_t
\end{align}$$
    * zgrubne oszacowanie _prawdopodobnego_ nowego $w{t+1}=w_t-\gamma_t p_{t+1}$
    * gradient liczony w nowym miejscu — rozszerzenie momentum z _ekstrapolacją_
* większe $\beta$ powoduje wolniejszą reakcję na zmianę powierzchni błędu
* można pokazać, że NAG przyspiesza uczenie dla wypukłych funkcji i (bardzo) dobrze dobranych parametrów
* momentum też przyspiesza, ale jedynie na kwadratowych powierzchniach funkcji błędu
* nie ma teoretycznych wyników pokazujących przyspieszenie NAG dla sieci neuronowych, chociaż często takie obserwujemy
  * dla sieci neuronowych zwykle NAG zachowuje się równie dobrze jak momentum
* przyspieszenie oraz wygładzanie szumu wpływa na lepsze zachowanie algorytmów momentum i NAG

## SGD a Momentum a NAG
<img style="float: right;" src="ml_figures/opt2.gif" width=450>
$$
\begin{alignat}{5}
w_{t+1}&=w_t-\gamma_t\nabla{}L_i(w_t)\\
w_{t+1}&=w_t - \gamma_t{}\nabla L_i(w_t) + \beta(w_t-w_{t-1})\\
w_{t+1}&=w_t - \gamma_t(1+\beta)\nabla L_i(w_t) - \gamma\beta^2p_t
\end{alignat}
$$

[wizualizacje algorytmów [deeplearning.ai]](https://www.deeplearning.ai/ai-notes/optimization/index.html)